# Exemplo de Extração DataJud/CNJ

Este notebook demonstra o fluxo completo de extração de dados de processos judiciais usando o sistema DataJud/CNJ.

## Objetivos

1. Configurar o ambiente de extração
2. Validar números de processos (formato CNJ)
3. Extrair dados de 5 processos-piloto
4. Validar e analisar os dados extraídos
5. Gerar relatório com as 14 respostas padrão CNJ

In [ ]:
# Importar bibliotecas necessárias
import sys
import os
from pathlib import Path

# Adicionar diretório raiz ao path
sys.path.insert(0, os.path.abspath("."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Importar módulos do projeto
from extrair_datajud import DataJudExtractor
from validacao_cnj import ValidadorCNJ

# Configurar estilo dos gráficos
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

print("✓ Bibliotecas importadas com sucesso")

## 1. Configuração Inicial

Vamos criar os diretórios necessários e configurar o extrator.

In [ ]:
# Criar diretórios se não existirem
Path("dados").mkdir(exist_ok=True)
Path("resultados").mkdir(exist_ok=True)
Path("logs").mkdir(exist_ok=True)

print("✓ Diretórios criados")

# Inicializar extrator
# NOTA: Configure suas credenciais no arquivo .env
extractor = DataJudExtractor()

print("✓ Extrator inicializado")

## 2. Definir Processos-Piloto

Vamos trabalhar com 5 processos de exemplo para teste inicial.

In [ ]:
# Números de processos para teste (formato CNJ)
# IMPORTANTE: Substitua pelos números reais de processos que deseja consultar
processos_piloto = [
    "0000001-01.2024.8.00.0001",
    "0000002-01.2024.8.00.0001",
    "0000003-01.2024.8.00.0001",
    "0000004-01.2024.8.00.0001",
    "0000005-01.2024.8.00.0001"
]

print(f"Processos selecionados: {len(processos_piloto)}")
for i, num in enumerate(processos_piloto, 1):
    print(f"  {i}. {num}")

## 3. Validação dos Números de Processos

Antes de fazer a extração, vamos validar se os números estão no formato correto.

In [ ]:
# Validar cada número
validacoes = []

print("Validando números de processos...\n")

for numero in processos_piloto:
    valido, mensagem = ValidadorCNJ.validar_formato(numero)
    validacoes.append({
        "numero": numero,
        "valido": valido,
        "mensagem": mensagem
    })
    
    status = "✓" if valido else "✗"
    print(f"{status} {numero}: {mensagem}")

# Criar DataFrame com validações
df_validacoes = pd.DataFrame(validacoes)
print(f"\nTotal válidos: {df_validacoes['valido'].sum()}/{len(processos_piloto)}")

## 4. Extração de Dados

Agora vamos extrair os dados dos processos válidos. 

**NOTA**: Esta seção requer credenciais válidas da API DataJud.

In [ ]:
# Filtrar apenas processos válidos
processos_validos = df_validacoes[df_validacoes["valido"] == True]["numero"].tolist()

print(f"Processos válidos para extração: {len(processos_validos)}")

# IMPORTANTE: Descomente as linhas abaixo quando tiver credenciais configuradas
# df_resultados = extractor.processar_lote(processos_validos)
# print(f"\nDados extraídos: {len(df_resultados)} processos")

# Para demonstração, vamos criar dados de exemplo
print("\n⚠️  MODO DEMONSTRAÇÃO: Usando dados de exemplo")

dados_exemplo = []
for i, numero in enumerate(processos_validos, 1):
    dados_exemplo.append({
        "1_numero_processo": numero,
        "2_classe": "Procedimento Comum Cível",
        "3_assunto": "Direito Civil",
        "4_data_ajuizamento": f"2024-0{i}-15",
        "5_orgao_julgador": f"1ª Vara Cível",
        "6_magistrado": f"Juiz(a) Exemplo {i}",
        "7_valor_causa": 10000 + (i * 5000),
        "8_partes_polo_ativo": f"Autor {i}",
        "9_partes_polo_passivo": f"Réu {i}",
        "10_movimentacoes": [{"data": f"2024-0{i}-20", "descricao": "Distribuição"}],
        "11_data_ultima_movimentacao": f"2024-0{i}-25",
        "12_situacao_processo": "Em andamento",
        "13_data_transito_julgado": "",
        "14_resultado": "Pendente"
    })

df_resultados = pd.DataFrame(dados_exemplo)
print(f"\nDataFrame criado com {len(df_resultados)} registros")

## 5. Análise dos Dados Extraídos

Vamos analisar os dados obtidos.

In [ ]:
# Exibir primeiras linhas
print("Primeiras linhas dos dados extraídos:\n")
display(df_resultados.head())

# Informações sobre o DataFrame
print("\nInformações do DataFrame:")
print(df_resultados.info())

In [ ]:
# Estatísticas descritivas
print("Estatísticas dos valores de causa:\n")
print(df_resultados["7_valor_causa"].describe())

# Visualização
plt.figure(figsize=(10, 6))
plt.bar(range(len(df_resultados)), df_resultados["7_valor_causa"])
plt.xlabel("Processo")
plt.ylabel("Valor da Causa (R$)")
plt.title("Valores de Causa dos Processos Piloto")
plt.xticks(range(len(df_resultados)), [f"P{i+1}" for i in range(len(df_resultados))])
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.savefig("resultados/valores_causa_piloto.png", dpi=150, bbox_inches="tight")
plt.show()

print("\n✓ Gráfico salvo em: resultados/valores_causa_piloto.png")

## 6. Estrutura das 14 Respostas CNJ

Os dados extraídos seguem a estrutura padrão do CNJ com 14 campos principais:

In [ ]:
# Listar os 14 campos
campos_cnj = [
    "1. Número do Processo",
    "2. Classe Processual",
    "3. Assunto",
    "4. Data de Ajuizamento",
    "5. Órgão Julgador",
    "6. Magistrado",
    "7. Valor da Causa",
    "8. Partes - Polo Ativo",
    "9. Partes - Polo Passivo",
    "10. Movimentações",
    "11. Data da Última Movimentação",
    "12. Situação do Processo",
    "13. Data do Trânsito em Julgado",
    "14. Resultado"
]

print("14 CAMPOS PADRÃO CNJ:\n")
for campo in campos_cnj:
    print(f"  {campo}")

# Verificar campos presentes
colunas_presentes = [col for col in df_resultados.columns if col.startswith(("1_", "2_", "3_", "4_", "5_", "6_", "7_", "8_", "9_", "10_", "11_", "12_", "13_", "14_"))]
print(f"\nCampos extraídos: {len(colunas_presentes)}/14")

## 7. Exportar Resultados

Vamos salvar os resultados em diferentes formatos.

In [ ]:
# Criar timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Salvar em Excel
arquivo_excel = f"resultados/extracao_piloto_{timestamp}.xlsx"
df_resultados.to_excel(arquivo_excel, index=False, sheet_name="Dados Extraídos")
print(f"✓ Dados salvos em: {arquivo_excel}")

# Salvar em CSV
arquivo_csv = f"resultados/extracao_piloto_{timestamp}.csv"
df_resultados.to_csv(arquivo_csv, index=False, encoding="utf-8-sig")
print(f"✓ Dados salvos em: {arquivo_csv}")

# Salvar validações
arquivo_validacao = f"resultados/validacao_piloto_{timestamp}.xlsx"
df_validacoes.to_excel(arquivo_validacao, index=False, sheet_name="Validações")
print(f"✓ Validações salvas em: {arquivo_validacao}")

## 8. Relatório Sumário

Vamos gerar um relatório sumário da extração.

In [ ]:
print("=" * 80)
print("RELATÓRIO DE EXTRAÇÃO - PROCESSOS PILOTO")
print("=" * 80)
print()
print(f"Data/Hora: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print(f"Total de processos: {len(processos_piloto)}")
print(f"Processos válidos: {len(processos_validos)}")
print(f"Processos extraídos: {len(df_resultados)}")
print()
print("ESTATÍSTICAS:")
print(f"  - Valor médio das causas: R$ {df_resultados['7_valor_causa'].mean():.2f}")
print(f"  - Valor total das causas: R$ {df_resultados['7_valor_causa'].sum():.2f}")
print()
print("SITUAÇÃO DOS PROCESSOS:")
print(df_resultados["12_situacao_processo"].value_counts())
print()
print("=" * 80)
print("✓ Extração concluída com sucesso!")

## Próximos Passos

1. **Configurar credenciais reais**: Adicione sua chave API no arquivo `.env`
2. **Testar com processos reais**: Substitua os números de exemplo por processos reais
3. **Expandir análise**: Use os dados extraídos para análises jurimétricas
4. **Automatizar**: Configure execução periódica com logs
5. **Validar dados**: Verifique a qualidade e completude dos dados extraídos

## Observações Importantes

- Este é um notebook de demonstração com dados fictícios
- Para uso em produção, configure credenciais válidas da API DataJud
- Respeite os limites de requisições da API
- Sempre valide os números de processos antes da extração
- Mantenha logs detalhados de todas as operações